In [ ]:
pip install topicmodeltuner

In [ ]:
from topictuner import TopicModelTuner as TMT
from sklearn.datasets import fetch_20newsgroups

Get 20NewsGroup data

In [ ]:
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

Create a TMT instance from scratch

In [ ]:
tmt = TMT(verbose=2) # verbose turns tqdm on

Alternatively, you can create one from an existing BERTopic instance by calling 

newTMT = TMT.wrapBERTopicModel(\<your BERTopic model\>)

Create the embeddings.

In [ ]:
tmt.createEmbeddings(docs)

Then reduce them to 5 features ala BERTopic by calling TMT.reduce()

In [ ]:
tmt.reduce()

Now we can explore different HDBSCAN settings for this instance of the UMAP reductions.

There are four search functions - `randomSearch`, `psuedoGridSearch`, `simpleSearch`, and `gridSearch`. All four take a list of `min_cluster_sizes` as the first argument. The first four search methods taketwo lists as input. The first is the `min_cluster_size` arguments. The second argument for `randomSearch` and `psuedoGridSearch` is a list of floats where each values <= 1. In these first two search methods values for the second list are used to multiply with a corresponding value from the first list to produce a valid integer value for the `sample_size` parameter. Examples follow below.

For simpleSearch the second array is a list of min_sample values that corresponds 1:1 with the first list of `min_cluster_size` values. With gridSearch you just provide the values of the `min_cluster_sizes` you are interested in and then searches are run for every possible sample_size value.

Typically you will do an initial narrowing of the search using `randomSearch`. `randomSearch` has a third parameter for the number of searches that will be run from the values provided and defaults to 20. All the other searches will exhaust whatever parameters are passed. Once you've narrowed down with `randomSearch` you can further narrow down with the remaining searches. Be aware that there can be a non-linear relationship between different setting pairs. For example, just because settings of 125,30 provide better results than say, 200,100, it may be that 215,22 work best. Running these searches is reasonably cheap computationally speaking, and `randomSearch` will generally give you a "lay of the land" so you can make judgements about where to focus your efforts.

Each search method returns a sorted DataFrame with that passes results. All the results for any session are aggregated in the `TMT.ResultsDF` DataFrame. `TMT.summarizeResults(`) will summarize the ResultsDF DataFrame or you can pass it your interim results and then it will summarize those for you. See below for examples.

In the next cell 20 random searches will be run using randomly chosen `min_cluster_size` values between 120 and 150. The corresponding `sample_size` values will be generated by randomly selecting one of the percentage multipliers and calculating the appropriate `sample_size` value for that `min_cluster_size`.

In [ ]:
lastRunResultsDF = tmt.randomSearch([*range(30,151)], [.1, .25, .5, .75, 1])

`lastRunResultsDF` is a dataframe containing the results from the 20 searches just completed

TMT.visualizeSearch produces a plotly parallel coordinates graph. You can pass it TMT.ResultsDF to get a view of all the searches, or pass it the results from a particular search.

In [ ]:
lastRunResultsDF

The cell below runs psuedoGridSearch for all values from 62 to 70 with `sample_size` values at 10% to 100% of each of the `min_cluster_size` values.

In [ ]:
lastRunResultsDF = tmt.psuedoGridSearch([*range(62,71)], [x/100 for x in range(10,101,10)])

The immediate results of the above search will be contained in lastRunResultsDF. You can see a summary DataFrame using the `TMT.summarizeResult()` method.

In [ ]:
tmt.summarizeResults(lastRunResultsDF)

If you call summarizeResults() without explicitly passing a DataFrame it will use the internal TMT.ResultsDF DataFrame which contains all the search results run this session.

In [ ]:
tmt.summarizeResults()

`TMT.summarizeResults` sorts a results table by number_of_cluster and selects the 'best' value for that number of clusters by choosing the one with the least uncategorized results. You can sort the DataFrame by whatever value interests you.

In [ ]:
tmt.summarizeResults(lastRunResultsDF).sort_values(by=['number_uncategorized'])

There are further visual aids to help you find the best parameters for your model. `TMT.visualizeSearch`, like `summarizeResults`, can be called against a subset of the searches performed or against the entire search space.

In [ ]:
tmt.visualizeSearch(lastRunResultsDF).show()

TMT can generate a scatterplot of your embeddings overlayed with the clustering of a given set of parameters. This can assist in deciding how many clusters to select for your model.

To accomplish this, first you must create a 2D reduction of the embeddings suitable for a 2D scatterplot. The default 2D reduction method is UMAP, but you can also specify TSNE. (You can also use your own 2D reduction by simply setting `TMT.viz_reduction`.)

In [ ]:
tmt.createVizReduction('TSNE')

Once there is a 2D representation of the embeddings you can view them using different values for the HDBSCAN parameters.

In [ ]:
tmt.visualizeEmbeddings(131,78).show()

You can save your TMT model with TMT.save()

In [ ]:
tmt.save('temp')

And restore it using TMT.load()

In [ ]:
tmt2 = TMT.load('temp')

Once you have determined parameters that work for your text, TMT can manufacture a BERTopic model. Note in this example we pass BERTopic the embeddings created earlier - no need to have BERTopic re-run them. (Although BERTopic will rerun UMAP).

In [ ]:
bt1 = tmt2.getBERTopicModel(131, 24)
bt1.fit_transform(tmt2.docs, tmt2.embeddings)
bt1.get_topic_info()